# installing Jpype and Jsprit Libraries

In [1]:
!pip install jpype
!pip install up_jsprit
!pip install unified_planning

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.5/635.5 kB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26304 sha256=560130c2723ffc3bd39349752c8b64f46fb4222d538df6e8c22354229583

# Importing jpype to run

In [2]:
import jpype
import jpype.imports
from jpype.types import *
import os
from up_jsprit import config #for local distibution
from up_jsprit.utils import generate_map, get_distance_and_time_from_graphhopper #for local distibution
from typing import Callable, IO, Optional
import unified_planning as up
from unified_planning import engines
import random

### import Java libraries from graphhopper

In [3]:
env = up.environment.get_environment()
env.factory.add_engine('jspritplanner', 'up_jsprit', 'JSpritSolver')

#env = problem.environment

# Start the JVM and set classpaths
jpype.getDefaultJVMPath()

if not jpype.isJVMStarted():
    jpype.startJVM(convertStrings=False)
    print("JVM Started")
if jpype.isJVMStarted():
    print("JVM already started")

# Add the paths to the JAR files
for jar_file in config.JAR_FILES:
    jpype.addClassPath(jar_file)

#Import necessary Java From Jsprit classes using JClass
VehicleRoutingProblem = jpype.JClass("com.graphhopper.jsprit.core.problem.VehicleRoutingProblem")
VehicleImpl = jpype.JClass("com.graphhopper.jsprit.core.problem.vehicle.VehicleImpl")
VehicleType = jpype.JClass("com.graphhopper.jsprit.core.problem.vehicle.VehicleTypeImpl")
Location = jpype.JClass("com.graphhopper.jsprit.core.problem.Location")
Shipment = jpype.JClass("com.graphhopper.jsprit.core.problem.job.Shipment")
Plotter = jpype.JClass("com.graphhopper.jsprit.analysis.toolbox.Plotter")
SelectBest = jpype.JClass("com.graphhopper.jsprit.core.algorithm.selector.SelectBest")
SolutionPrinter = jpype.JClass("com.graphhopper.jsprit.core.reporting.SolutionPrinter")
GraphStreamViewer = jpype.JClass("com.graphhopper.jsprit.analysis.toolbox.GraphStreamViewer")
AlgorithmSearchProgressChartListener = jpype.JClass("com.graphhopper.jsprit.analysis.toolbox.AlgorithmSearchProgressChartListener")
Jsprit = jpype.JClass("com.graphhopper.jsprit.core.algorithm.box.Jsprit")
Coordinate = jpype.JClass("com.graphhopper.jsprit.core.util.Coordinate")
VehicleRoutingTransportCostsMatrix = jpype.JClass("com.graphhopper.jsprit.core.util.VehicleRoutingTransportCostsMatrix")
VehicleRoutingTransportCostsMatrix = jpype.JClass("com.graphhopper.jsprit.core.util.VehicleRoutingTransportCostsMatrix")
#VehicleRoutingAlgorithm = jpype.JClass("com.graphhopper.jsprit.core.problem.VehicleRoutingAlgorithm")
# Import necessary Java classes for printing the solution
PrintWriter = jpype.JClass('java.io.PrintWriter')
File = jpype.JClass('java.io.File')
PrintStream = jpype.JClass('java.io.PrintStream')
System = jpype.JClass('java.lang.System')

JVM Started
JVM already started


In [4]:
#adding other library
Service = jpype.JClass("com.graphhopper.jsprit.core.problem.job.Service")


In [5]:
VrpXMLWriter=jpype.JClass("java.io.OutputStream")

### The attempt to build a sample simple model to run from the Java directly

In [6]:
#STEP 0 - BUILD VEHICLE ROUTING PROBLEM

vrpbuilder = VehicleRoutingProblem.Builder.newInstance()
#Building the model of two vehicle types


WEIGHT_INDEX = 0;
vehicleTypeBuilder = VehicleType.Builder.newInstance("type").addCapacityDimension(WEIGHT_INDEX,100).setCostPerDistance(1).setCostPerTime(2).setFixedCost(2);
vehicleType1 = vehicleTypeBuilder.build();


vehicleTypeBuilder2 = VehicleType.Builder.newInstance("type").addCapacityDimension(WEIGHT_INDEX,1000).setCostPerDistance(4).setCostPerTime(4).setFixedCost(4);
vehicleType2 = vehicleTypeBuilder.build();

#type = VehicleTypeImpl.Builder.newInstance("type").addCapacityDimension(0, 1).setCostPerDistance(1).setCostPerTime(2).build();

# setting initial and final location of the vehicles
vehicleBuilder = VehicleImpl.Builder.newInstance("vehicle1");
vehicleBuilder.setStartLocation(Location.newInstance("0"));
vehicleBuilder.setEndLocation(Location.newInstance("0"));
vehicleBuilder.setType(vehicleType1);
vehicle1 = vehicleBuilder.build();


vehicleBuilder = VehicleImpl.Builder.newInstance("vehicle2");
vehicleBuilder.setStartLocation(Location.newInstance("0"));
vehicleBuilder.setEndLocation(Location.newInstance("0"));
vehicleBuilder.setType(vehicleType1);
vehicle2 = vehicleBuilder.build();

vehicleBuilder = VehicleImpl.Builder.newInstance("vehicle3");
vehicleBuilder.setStartLocation(Location.newInstance("0"));
vehicleBuilder.setEndLocation(Location.newInstance("0"));
vehicleBuilder.setType(vehicleType2);
vehicle3 = vehicleBuilder.build();

vehicleBuilder = VehicleImpl.Builder.newInstance("vehicle4");
vehicleBuilder.setStartLocation(Location.newInstance("0"));
vehicleBuilder.setEndLocation(Location.newInstance("0"));
vehicleBuilder.setType(vehicleType2);
vehicle4 = vehicleBuilder.build();

vehicleBuilder = VehicleImpl.Builder.newInstance("vehicle5");
vehicleBuilder.setStartLocation(Location.newInstance("0"));
vehicleBuilder.setEndLocation(Location.newInstance("0"));
vehicleBuilder.setType(vehicleType2);
vehicle5 = vehicleBuilder.build();

vehicleBuilder = VehicleImpl.Builder.newInstance("vehicle6");
vehicleBuilder.setStartLocation(Location.newInstance("0"));
vehicleBuilder.setEndLocation(Location.newInstance("0"));
vehicleBuilder.setType(vehicleType2);
vehicle6 = vehicleBuilder.build();

vehicleBuilder = VehicleImpl.Builder.newInstance("vehicle7");
vehicleBuilder.setStartLocation(Location.newInstance("0"));
vehicleBuilder.setEndLocation(Location.newInstance("0"));
vehicleBuilder.setType(vehicleType2);
vehicle7 = vehicleBuilder.build();

vehicleBuilder = VehicleImpl.Builder.newInstance("vehicle8");
vehicleBuilder.setStartLocation(Location.newInstance("0"));
vehicleBuilder.setEndLocation(Location.newInstance("0"));
vehicleBuilder.setType(vehicleType2);
vehicle8 = vehicleBuilder.build();

vehicleBuilder = VehicleImpl.Builder.newInstance("vehicle9");
vehicleBuilder.setStartLocation(Location.newInstance("0"));
vehicleBuilder.setEndLocation(Location.newInstance("0"));
vehicleBuilder.setType(vehicleType2);
vehicle9 = vehicleBuilder.build();

vehicleBuilder = VehicleImpl.Builder.newInstance("vehicle10");
vehicleBuilder.setStartLocation(Location.newInstance("0"));
vehicleBuilder.setEndLocation(Location.newInstance("0"));
vehicleBuilder.setType(vehicleType2);
vehicle10 = vehicleBuilder.build();

#STEP 1 - Adding the routing
#create a for loop for shipment 1000 locations

# Fill the Distance and Time Matrices:
for L in range(0,101):

    shipment_builder = Shipment.Builder.newInstance(str(L)).setPickupLocation(Location.newInstance("0"))
    shipment_builder.setDeliveryLocation(Location.newInstance(str(L)))
    shipment_builder.addSizeDimension(0,10);
    shipment_builder.setPickupServiceTime(1);
    shipment_builder.addPickupTimeWindow(0, 200);
    vrpbuilder.addJob(shipment_builder.build())



#s2 = Service.Builder.newInstance("2").addSizeDimension(0, 1).setLocation(Location.newInstance("2")).build();
#s3 = Service.Builder.newInstance("3").addSizeDimension(0, 1).setLocation(Location.newInstance("3")).build();
#s4 = Service.Builder.newInstance("4").addSizeDimension(0, 1).setLocation(Location.newInstance("4")).build();

#STEP 2 - BUILDING Customized cost matrix

costMatrixBuilder = VehicleRoutingTransportCostsMatrix.Builder.newInstance(True);


#creating a cost matrix using a for loop


# Fill the Distance and Time Matrices:
for i in range(0,101):
    for j in range(0,101):
        if i < j:  # Only compute for the upper triangle
            tm=random.randint(1, 10)
            ds=random.randint(1, 1000)
            costMatrixBuilder.addTransportDistance(str(i),str(j),ds )  #generate a random number for cost distance matix
            costMatrixBuilder.addTransportTime(str(i),str(j),tm)         #generate a random number for cost time matix
            print(f"{i}, {j}, {ds}, {tm}")

#costMatrixBuilder.addTransportDistance("0", "1", 10.0);
#costMatrixBuilder.addTransportDistance("0", "2", 20.0);
#costMatrixBuilder.addTransportDistance("0", "3", 5.0);
#costMatrixBuilder.addTransportDistance("1", "2", 4.0);
#costMatrixBuilder.addTransportDistance("1", "3", 1.0);
#costMatrixBuilder.addTransportDistance("2", "3", 2.0);

#costMatrixBuilder.addTransportTime("0", "1", 10.0);
#costMatrixBuilder.addTransportTime("0", "2", 20.0);
#costMatrixBuilder.addTransportTime("0", "3", 5.0);
#costMatrixBuilder.addTransportTime("1", "2", 4.0);
#costMatrixBuilder.addTransportTime("1", "3", 1.0);
#costMatrixBuilder.addTransportTime("2", "3", 2.0);


Streaming output truncated to the last 5000 lines.
0, 51, 208, 1
0, 52, 865, 10
0, 53, 180, 3
0, 54, 887, 6
0, 55, 653, 5
0, 56, 350, 4
0, 57, 39, 6
0, 58, 349, 8
0, 59, 510, 9
0, 60, 360, 5
0, 61, 599, 5
0, 62, 449, 8
0, 63, 311, 10
0, 64, 886, 3
0, 65, 973, 8
0, 66, 673, 8
0, 67, 203, 8
0, 68, 721, 8
0, 69, 359, 4
0, 70, 717, 6
0, 71, 214, 3
0, 72, 549, 10
0, 73, 224, 6
0, 74, 643, 3
0, 75, 822, 9
0, 76, 151, 7
0, 77, 971, 3
0, 78, 74, 6
0, 79, 472, 1
0, 80, 877, 8
0, 81, 152, 10
0, 82, 101, 4
0, 83, 455, 1
0, 84, 896, 6
0, 85, 39, 1
0, 86, 344, 6
0, 87, 200, 4
0, 88, 976, 4
0, 89, 473, 1
0, 90, 575, 2
0, 91, 160, 1
0, 92, 630, 2
0, 93, 287, 5
0, 94, 897, 1
0, 95, 473, 9
0, 96, 682, 6
0, 97, 986, 2
0, 98, 910, 3
0, 99, 732, 1
0, 100, 672, 7
1, 2, 780, 10
1, 3, 526, 2
1, 4, 21, 9
1, 5, 555, 7
1, 6, 524, 4
1, 7, 185, 4
1, 8, 526, 8
1, 9, 412, 2
1, 10, 305, 8
1, 11, 843, 4
1, 12, 456, 9
1, 13, 875, 2
1, 14, 821, 4
1, 15, 65, 7
1, 16, 894, 10
1, 17, 676, 2
1, 18, 812, 4
1, 19, 172, 9
1, 

In [7]:
# STEP 3- Run the problem and implement the solution

costMatrix = costMatrixBuilder.build();



In [8]:
#vrp = VehicleRoutingProblem.Builder.newInstance();
vrpbuilder.setRoutingCost(costMatrix);
vrpbuilder.addVehicle(vehicle1).addVehicle(vehicle2).addVehicle(vehicle3).addVehicle(vehicle4).addVehicle(vehicle5).addVehicle(vehicle6).addVehicle(vehicle7).addVehicle(vehicle8).addVehicle(vehicle9).addVehicle(vehicle10);


In [9]:
vrp = vrpbuilder.build()

In [10]:
# step  solving the algorithm through running

# get the algorithm out-of-the-box.
algorithm = Jsprit.createAlgorithm(vrp);


# and search a solution which returns a collection of solutions (here only one solution is constructed)
# Set the maximum number of iterations
algorithm.setMaxIterations(50)

solutions = algorithm.searchSolutions();


# Print the solution progress as function of the iteration
algorithm.getAlgorithmListeners().addListener(AlgorithmSearchProgressChartListener(os.path.join(config.OUTPUT_DIRECTORY, "solution_progress.png")))


# Retrieve the best solution
bestSolution = SelectBest().selectSolution(solutions)


#Printing the solution and the solution plot

In [11]:
print(vrp)

[fleetSize=INFINITE][#jobs=101][#vehicles=10][#vehicleTypes=1][transportCost=com.graphhopper.jsprit.core.util.VehicleRoutingTransportCostsMatrix@815b41f][activityCosts=com.graphhopper.jsprit.core.problem.cost.WaitingTimeCosts@5542c4ed]


In [12]:
print(bestSolution)

[costs=12483.0][routes=3][unassigned=0]


In [13]:
# Create a PrintWriter that writes directly to a file
with PrintWriter(File(os.path.join(config.OUTPUT_DIRECTORY, "problemsolve.txt"))) as writer:
            SolutionPrinter.print_(writer, vrp, bestSolution, SolutionPrinter.Print.VERBOSE) # Using the corrected print_ method






In [14]:
import os
print(os.getcwd())
!ls

from google.colab import files

files.download('/content/problemsolve.txt')



/content
problemsolve.txt  sample_data


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
VrpXMLWriter(vrp, solutions).write("output/problem-with-solution.xml");

TypeError: No matching overloads found for constructor java.io.OutputStream(com.graphhopper.jsprit.core.problem.VehicleRoutingProblem,java.util.ArrayList), options are:
	public java.io.OutputStream()


In [ ]:
# Plot the solution
solutionPlot = Plotter(vrp, bestSolution)
solutionPlot.plot(os.path.join(config.OUTPUT_DIRECTORY, "solution_progress.png"), "Best Solution")

In [ ]:
# Plot the solution
solutionPlot = Plotter(vrp, bestSolution)
solutionPlot.plot(os.path.join(config.OUTPUT_DIRECTORY, "solutionPlot.png"), "Best Solution")

In [ ]:
# Plot the solution
solutionPlot = Plotter(vrp, bestSolution)
solutionPlot.plot(os.path.join(config.OUTPUT_DIRECTORY, "solutionPlot.png"), "Best Solution")

In [ ]:
# Stop JVM
jpype.shutdownJVM()